In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

In [19]:
data=loadmat('ex3data1.mat')

In [20]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [21]:
X=data['X']
y=data['y']
X.shape,y.shape

((5000, 400), (5000, 1))

In [22]:
from sklearn.preprocessing import OneHotEncoder
ohc=OneHotEncoder(sparse=False)
y_onehot=ohc.fit_transform(y)
y_onehot.shape

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(5000, 10)

In [23]:
y[0],y_onehot[0,:]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

In [24]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [25]:
def forward_propagate(X,theta1,theta2):
    m=X.shape[0]
    a1=np.insert(X,0,np.ones(m),axis=1)
    z2=a1*theta1.T
    a2=np.insert(sigmoid(z2),0,np.ones(m),axis=1)
    z3=a2*theta2.T
    h=sigmoid(z3)
    return a1,z2,a2,z3,h

In [26]:
def cost(params,input_size,hidden_size,num_labels,X,y,Regularisation_param):
    m=X.shape[0]
    X=np.matrix(X)
    y=np.matrix(y)
    theta1=np.matrix(np.reshape(params[:hidden_size * (input_size+1)],(hidden_size,(input_size+1))))
    theta2=np.matrix(np.reshape(params[hidden_size * (input_size+1):],(num_labels,(hidden_size+1))))
    a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
    J=0
    for i in range(m):
        first=np.multiply(-y[i,:],np.log(h[i,:]))
        second=np.multiply((1-y[i,:]),np.log(1-h[i,:]))
        J+=np.sum(first-second)
    J=J/m
    J += (float(Regularisation_param) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    return J

In [74]:
input_size=400
hidden_size=25
num_labels=10
Regularisation_param=1
params=((np.random.random(size=hidden_size*(input_size+1)+num_labels*(hidden_size+1))-0.5)*0.25) #2*rand-randgenerate random matrix
m=X.shape[0]
X=np.matrix(X)
y=np.matrix(y)
theta1=np.matrix(np.reshape(params[:hidden_size * (input_size+1)],(hidden_size,(input_size+1))))
theta2=np.matrix(np.reshape(params[hidden_size * (input_size+1):],(num_labels,(hidden_size+1))))

In [75]:
theta1.shape,theta2.shape

((25, 401), (10, 26))

In [76]:
a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
a1.shape,z2.shape,a2.shape,z3.shape,h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [77]:
J=cost(params,input_size,hidden_size,num_labels,X,y_onehot,Regularisation_param)
print(J)

6.758564075720514


In [78]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z),sigmoid(1-z))

In [79]:
def backpropagation(params,input_size,hidden_size,num_labels,X,y,Regularisation_param):
    m=X.shape[0]
    X=np.matrix(X)
    y=np.matrix(y)
    theta1=np.matrix(np.reshape(params[:hidden_size * (input_size+1)],(hidden_size,(input_size+1))))
    theta2=np.matrix(np.reshape(params[hidden_size * (input_size+1):],(num_labels,(hidden_size+1))))
    a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
    J=0
    delta1=np.zeros(theta1.shape)
    delta2=np.zeros(theta2.shape)
    J=cost(params,input_size,hidden_size,num_labels,X,y,Regularisation_param)
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * Regularisation_param) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * Regularisation_param) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [81]:
J,grad=backpropagation(params,input_size,hidden_size,num_labels,X,y_onehot,Regularisation_param)

In [82]:
J,grad.shape

(6.758564075720514, (10285,))

In [85]:
from scipy.optimize import minimize
fmin=minimize(fun=backpropagation,x0=params,args=(input_size,hidden_size,num_labels,X,y_onehot,Regularisation_param),method='TNC',jac=True,options={'maxiter':50})
fmin

     fun: 0.6991139721094806
     jac: array([ 4.21791705e-03, -9.96759477e-06, -1.21371563e-05, ...,
       -3.80342066e-03, -8.17902917e-04,  2.05789515e-04])
 message: 'Max. number of function evaluations reached'
    nfev: 50
     nit: 10
  status: 3
 success: False
       x: array([-0.03414438, -0.04983797, -0.06068578, ...,  1.13782113,
       -0.2033825 , -2.15520601])

In [86]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=int64)

In [88]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print("accuracy = {0}%".format(accuracy * 100))

accuracy = 91.53999999999999%
